https://www.kaggle.com/competitions/oxford-102-flower-pytorch

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import cv2
import random
from skimage.io import imread
from sklearn.utils import class_weight
from collections import Counter
import time

In [ ]:
# Tensorflow libraries and modules
import itertools
import tensorflow as tf 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import models, optimizers, regularizers

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# path dataset
path_train='flower_data/train/'
path_val='flower_data/valid/'
path_test='flower_data/test/'

In [ ]:
# GPU
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
num_cat=[]
for x in os.listdir(path_train):
    num_cat.append(x)
num_cat[0:5]

In [ ]:
len(set(num_cat))

In [ ]:
# Opening JSON file
f = open('flower_data/cat_to_name.json')
# json to dictionary
data_class = json.load(f)
# Closing file
f.close()

In [ ]:
# dictionary classes
data_class['21']

In [ ]:
# make dataframe
train_folder_df=pd.DataFrame([data_class]).T
train_folder_df.columns=['class']

In [ ]:
train_folder_df.head(3)

In [ ]:
# plot some image examples
plt.figure(figsize = (19, 8))
i=1
for cl in range(12):
    im=train_folder_df.sample(n=1)
    ax=plt.subplot(2,6,i)
    path=path_train+im.index[0]+'/'
    #img = cv2.imread(path+random.choice(os.listdir(path)))
    img = imread(path+random.choice(os.listdir(path)))
    plt.imshow(img)
    plt.title(im['class'][0])
    i+=1
    

In [ ]:
# counter (checking balance data classes)
counter={}
for i, row in train_folder_df.iterrows():
    count_={row[0]:len(os.listdir(path_train+i[0]))}
    counter.update(count_)
    
counter_df=pd.DataFrame([counter]).T.reset_index()  
counter_df.columns=['class','count']
counter_df['class_num']=train_folder_df.index
counter_df.head(3)

In [ ]:
# min max database flower class
counter_df.describe().T[['min', 'max']]

In [ ]:
plt.figure(figsize = (19, 2))
plt.plot(counter_df.index, counter_df['count']);

## Convolutional Neural Network (CNN) TensorFlow

In [ ]:
# target_size
target_size = (32,32)
# number of subprocesses data loading
num_workers = 4
# samples per batch to load
batch_size = 32
# input_shape
input_shape = (target_size[0],target_size[1], 3)

### Data Generator

In [ ]:
# Datagen and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=None)

validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Train / Validation - Data Generator
train_generator = train_datagen.flow_from_directory(path_train,
                                 target_size=target_size,
                                 batch_size=batch_size,
                                 class_mode = 'categorical')

validation_generator = validation_datagen.flow_from_directory(path_val,
                                 target_size=target_size,
                                 batch_size=batch_size,
                                 class_mode = 'categorical')

# Test 
test_generator = test_datagen.flow_from_directory(path_test,
                                 target_size=target_size,
                                 shuffle = False,
                                 batch_size=batch_size,
                                 class_mode = 'categorical')

In [ ]:
all_values_class=[]
for i, row in counter_df.iterrows():
    for j in range(row[1]):
        all_values_class.append(row[2])
len(all_values_class)

In [ ]:
# class_weight - Unbalanced Data                       
#class_weights = dict(zip(np.unique(all_values_class), class_weight.compute_class_weight('balanced', np.unique(all_values_class),all_values_class)))
class_weights = None

In [ ]:
# Define optimizer
optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
# callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=6)
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',patience=4) 

In [ ]:
# plot output function
def plot_model_output(history, name='model'):
    
    history_dict = history.history
    loss_values = history_dict['loss']
    val_loss_values = history_dict['val_loss']
    accuracy_values = history_dict['accuracy']
    val_accuracy_values = history_dict['val_accuracy']
    
    fig = plt.figure(figsize=(19,3))
    
    plt.subplot(1, 2, 1)
    plt.suptitle(name, fontsize=18)
    plt.title('loss')
    epoch = range(1,len(loss_values)+1)
    plt.plot(epoch,loss_values, '--',label='loss')
    plt.plot(epoch,val_loss_values, '--',label='val_loss')
    plt.legend()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    
    plt.subplot(1, 2, 2)
    plt.suptitle(name, fontsize=18)
    plt.title('accuracy')
    epoch = range(1,len(loss_values)+1)
    plt.plot(epoch,accuracy_values, '--',label='accuracy')
    plt.plot(epoch,val_accuracy_values, '--',label='val_accuracy')
    plt.legend()
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.show()

### Model

In [ ]:
# Model VGG19
model_vgg19 = tf.keras.applications.VGG19(
    weights='imagenet',
    include_top = False,
    input_shape = input_shape)

for layer in model_vgg19.layers:
    layer.trainable=False
    
x = model_vgg19.output
x = Flatten()(x)
x = Dense(128,activation='relu')(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.2)(x)

# output layer
output_vgg19 = Dense(102, activation='softmax')(x)
# model
model_vgg19 = tf.keras.Model(inputs=model_vgg19.input, outputs=output_vgg19)
# name
model_vgg19._name = "model_vgg19"
# summary
#model_vgg19.summary()

In [ ]:
if os.path.exists('models/'):
    pass
else:
    os.makedirs('models/')

In [ ]:
# run models fuction

filepath=os.getcwd()+'/models/'
def run_model(model):
    
    # save best model (callback)
    modelCheckpoint = ModelCheckpoint(filepath+'{}.h5'.format(model.name), save_best_only = True)
    
    # Compile the model
    model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
    
    # Fit the model
    history = model.fit_generator(generator=train_generator,
                              epochs = epochs,
                              #steps_per_epoch=300,
                              callbacks=[early_stopping, lr, modelCheckpoint],
                              class_weight=class_weights,
                              validation_data = validation_generator)
    
    print(history.history.keys())
    
    return model, history

In [ ]:
# Run model
epochs=10
max_accuracy=[]
custom_model, history_custom = run_model(model_vgg19)
max_accuracy.append(('{}.h5'.format(custom_model.name),np.max(history_custom.history['accuracy'])))

In [ ]:
# plot model ouputs
plot_model_output(history_custom, 'custom_model')